<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [4]:
# импорт библиотеки
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [145]:
# текст запроса
query_3_1 = f'''
SELECT
    COUNT(*)
FROM
    vacancies;
'''

In [146]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, engine)
df_3_1

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [147]:
# текст запроса
query_3_2 = f'''
SELECT
    COUNT(*)
FROM
    employers;
'''

In [148]:
# результат запроса
df_3_2= pd.read_sql_query(query_3_2, engine)
df_3_2

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [149]:
# текст запроса
query_3_3 = f'''
SELECT 
    COUNT(*) 
FROM
    areas;
'''

In [150]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, engine)
df_3_3

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [151]:
# текст запроса
query_3_4 = f'''
SELECT
    COUNT(*)
FROM
    industries;
'''

In [152]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, engine)
df_3_4

,count
0,294


**Выводы**

На основе предварительного анализа данных можно сделать следующие выводы:

* Количество вакансий примерно в 2 раза больше, чем количество работодателей, 
указанных в БД. 
* По количеству регионов можно судить, что это не субъекты РФ, а города.
* Количество сфер деятельности значительно меньше, чем количество компаний, что является ожидаемым результатом. 
Вероятно, некоторые компании занимаются несколькими видами деятельности. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [153]:
# текст запроса
# подсчет количества вакансий в таблице vacancies, группировка результатов запроса по уникальным значениям региона, сортировка по убыванию
query_4_1 = f'''
SELECT 
    COUNT(vacancies.id) AS cnt,
    areas.name
FROM 
    vacancies, areas
WHERE
    vacancies.area_id = areas.id
GROUP BY
    areas.name
ORDER BY 
    cnt DESC;
'''


In [154]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, engine)
df_4_1

,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [155]:
# текст запроса
# подсчет количества вакансий, где указана зп хотя бы в одном поле
query_4_2 = f'''
SELECT
    COUNT(id) AS cnt
FROM
    vacancies
WHERE
    salary_from IS NOT NULL 
    OR salary_to IS NOT NULL;
'''

In [156]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, engine)
df_4_2

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [157]:
# текст запроса
# нахождение средних значений границы зарплатной вилки и их округление до целого
query_4_3 = f'''
SELECT
    ROUND(AVG(salary_from)) AS salary_from_avg,
    ROUND(AVG(salary_to)) AS salary_to_avg
FROM
    vacancies;
'''

In [158]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, engine)
df_4_3

,salary_from_avg,salary_to_avg
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [159]:
# текст запроса
# вывод количества вакансий для каждого сочетания schedule и employment, сортировка по убыванию
query_4_4 = f'''
SELECT
    COUNT(id) AS cnt, 
    employment, 
    schedule
FROM 
    vacancies
GROUP BY
    employment, schedule
ORDER BY
    cnt DESC;
'''

In [160]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, engine)
df_4_4

,cnt,employment,schedule
0,35367,Полная занятость,Полный день
1,7802,Полная занятость,Удаленная работа
2,1593,Полная занятость,Гибкий график
3,1312,Частичная занятость,Удаленная работа
4,940,Полная занятость,Сменный график
5,569,Стажировка,Полный день
6,367,Полная занятость,Вахтовый метод
7,347,Частичная занятость,Полный день
8,312,Частичная занятость,Гибкий график
9,141,Проектная работа,Полный день


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [161]:
# текст запроса
# вывод количества вакансий в зависимости от опыта работы в порядке возрастания
query_4_5 = f'''
SELECT 
    COUNT(id) AS cnt, 
    experience
FROM 
    vacancies
GROUP BY
    experience
ORDER BY
    cnt ASC;
'''

In [162]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, engine)
df_4_5

,cnt,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

**Выводы**

На основе анализа данных в данном юните можно сделать следующие выводы:

* Наибольшее количество вакансий расположено в Москве, на втором месте распологается Санкт-Петербург.
* Не все работадатели предоставляют информацию о зарплатной вилке. 
* Средние значения зарплатной вилки - от 71065 руб. до 110537 руб. 
* Большинство предложений предполагает полный рабочий день и полную занятость
* Большинство вакансий требует от 1 до 3 лет опыта работы. 


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [164]:
# текст запроса
# объединение таблиц vacancies и employers по ключам, груупировка по наименованию работодателя, 
# сортировка по количеству вакансий и выведение первых 5 вакансий
query_5_1 = f'''
SELECT
    e.name, 
    COUNT(v.id) AS cnt
FROM
    vacancies v
JOIN
    employers e ON v.employer_id = e.id
GROUP BY
    e.name
ORDER BY
    cnt DESC
LIMIT 5;
'''

In [165]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, engine)
df_5_1

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [166]:
# текст запроса
# объединение таблиц areas, employer, vacancies по ключам
# подсчет количества работадателей и вакансий в каждом регионе
# группировка по региону
# сортировка по количеству работадателей
# среди регионов, в которых нет вакансий, нахождение того, в котором наибольшее количество работодателей.
query_5_2 = f'''
SELECT 
    a.name,
    COUNT(DISTINCT e.id) AS num_employers,
    COUNT(DISTINCT v.id) AS num_vacancies
FROM 
    areas a
LEFT JOIN 
    employers e ON a.id = e.area
LEFT JOIN 
    vacancies v ON a.id = v.area_id
WHERE 
    v.id is NULL
GROUP BY 
    a.id
ORDER BY
    num_employers DESC
LIMIT 1;
'''

In [167]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, engine)
df_5_2

,name,num_employers,num_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [169]:
# текст запроса
# объединение таблиц employers и vacancies по ключам
# группировка по работодателю
# сортировка по количеству регионов, в которых публикуются вакансии
# вывод работадателя и количество регионов
query_5_3 = f'''
SELECT 
    e.name AS employer_name,
    COUNT(DISTINCT v.area_id) AS num_regions
FROM 
    employers e
JOIN 
    vacancies v ON e.id = v.employer_id
GROUP BY 
    e.name
ORDER BY
    num_regions DESC;
'''

In [170]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, engine)
df_5_3

,employer_name,num_regions
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [171]:
# текст запроса
# объединение таблиц employers, employers_industries, industries по ключам
# вывод количества работодателей, у которых не указана сфера деятельности
query_5_4 = f'''
SELECT 
    COUNT(*) 
FROM 
    employers e
LEFT JOIN 
    employers_industries e_i ON e.id = e_i.employer_id
LEFT JOIN 
    industries i ON i.id = e_i.industry_id
WHERE 
    e_i.industry_id IS NULL OR e_i.industry_id = '';
'''

In [172]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, engine)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [173]:
# текст запроса
# объединение таблиц employers и employers_industries по ключам
# группировка по id
# указание условия о 4 видах деятельности
# сортировка по имени в алфавитном порядке
# вывод первых 3 компаний
query_5_5 = f'''
SELECT
    e.name, 
    COUNT(e_i.industry_id) AS count_industry
FROM
    employers e
JOIN
    employers_industries e_i ON e_i.employer_id = e.id
GROUP BY
    e.id
HAVING
    COUNT(e_i.industry_id) = 4
ORDER BY
    e.name
LIMIT 3;
'''

In [174]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, engine)
df_5_5

,name,count_industry
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [2]:
# текст запроса
# объединение таблиц employers, employers_industries, industries по ключам
# вывод количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения
query_5_6 = f'''
SELECT
    COUNT(DISTINCT e.id)
FROM
    employers e
JOIN
    employers_industries e_i ON e_i.employer_id = e.id
JOIN
    industries i ON i.id = e_i.industry_id
WHERE
    LOWER(i.name) ~ 'разработк.*программн.*обеспечен';
'''

In [6]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, engine)
df_5_6

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [128]:
# URL страницы Википедии со списком городов-миллионников России
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

# HTTP-запрос на получение содержимого страницы
response = requests.get(url)

# разбор содержимого страницы с помощью BeautifulSoup
page = BeautifulSoup(response.text, 'html.parser')

# находение в разметке таблицы с классом "standard" и преобразуем её в строку
cities_here = str(page.find('table', class_="standard"))

# разделение строки по фрагменту 'title="' и взятие всех элементов начиная с третьего
cities_here = cities_here.split('title="')[3:]

# создание пустого списока для городов
cities = []

# прохождение по всем элементам, полученным в предыдущем шаге
for i in cities_here:
    # разделение каждого элемента по '">', взятие первой части и добавление в список городов
    cities.append(i.split('">')[0])

# пустой кортеж
cit = ()

# преобразование списка городов в кортеж
cit = tuple(cities)

# вывод кортежа городов
print(cit)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [131]:
# текст запроса
# запрос с использованием общей таблицы, названной 'region'
# объединение таблиц vacancies, areas, employers по ключам
# условие: название работодателя 'Яндекс'
# группировка по имени региона
# фильтр для включения только определенных регионов, заданных в кортеже 'cit'
# основной запрос для выбора данных из 'region'
# специальная строка для итогового подсчета 'Total'
# сортировка по количеству вакансий
query_5_7 = f'''
WITH region AS
    (SELECT
        a.name AS region, 
        COUNT(v.id) AS vacancies
    FROM
        employers e
    JOIN
        vacancies v on e.id = v.employer_id
    JOIN
        areas a on v.area_id = a.id
    WHERE
        e.name = 'Яндекс'
    GROUP BY
        a.name
    HAVING a.name IN {cit})
SELECT
    region,
    vacancies
FROM
    region
UNION ALL
SELECT 
    'Total', 
    SUM(vacancies)  
FROM
    region
ORDER BY
    vacancies;
'''

In [176]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, engine)
df_5_7

,region,vacancies
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

**Выводы**

На основе анализа данных в данном юните можно сделать следующие выводы:

* Яндекс находится на первом месте по количеству размещенных вакансий, Газпром нефть - на пятом. 
* Среди регионов, в которых нет вакансий, Россия - регион, в котором наибольшее количество работодателей.
* Яндекс публикует больше всего вакансий в различных регионах.
* 8419 из 23501 работодателей не указали свою сферу деятельности.
* У 3553 из 23501 работодателей указана в качестве сферы деятельности указана Разработка программного обеспечения.
* Москва - лидер среди городов-миллионников по количеству размещенных вакансий от Яндекса. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [36]:
# текст запроса
# подсчет вакансий, которые содержат слова 'data' или 'данн', используя регулярные выражения
query_6_1 = f'''
SELECT
    COUNT(*) AS cnt
FROM
    vacancies
WHERE
    name ~* '.*data.*'
    OR name ~* '.*данн.*';
'''

In [177]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, engine)
df_6_1

,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [46]:
# текст запроса
# # подсчет вакансий для джунов, относящиейся к DS, используя регулярные выражения
query_6_2 = f'''
SELECT
    COUNT(*) as cnt
FROM
    vacancies
WHERE
    (name ~* '.*data.*scientist.*' 
    OR name ~* '.*data.*science.*'
    OR name ~* '.*исследовател.*данн.*'
    OR name ~* '.*machine.*learning.*'
    OR (name ~ '.*ML.*' AND name !~*'.*HTML.*')
    OR name ~* '.*машинн.*обучен.*')
    AND (name ~* '.*junior.*'
    OR experience ~* '.*нет.*опыта.*'
    OR employment ~* '.*стажировка.*');
'''

In [178]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, engine)
df_6_2

,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
# # подсчет вакансий, относящиейся к DS, требующие знания SQL или postgres, используя регулярные выражения

query_6_3 = f'''
SELECT
    COUNT(*) as cnt
FROM
    vacancies
WHERE
    (name ~* '.*data.*scientist.*' 
    OR name ~* '.*data.*science.*'
    OR name ~* '.*исследовател.*данн.*'
    OR name ~* '.*machine.*learning.*'
    OR (name ~ '.*ML.*' AND name !~ '.*HTML.*')
    OR name ~* '.*машинн.*обучен.*')
    AND (key_skills ~* '.*SQL.*'
    OR key_skills ~* '.*postgres.*');
'''

In [179]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, engine)
df_6_3

,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [48]:
# текст запроса
# # подсчет вакансий, относящиейся к DS, требующие знания python, используя регулярные выражения

query_6_4 = f'''
SELECT
    COUNT(*) as cnt
FROM
    vacancies
WHERE
    (name ~* '.*data.*scientist.*' 
    OR name ~* '.*data.*science.*'
    OR name ~* '.*исследовател.*данн.*'
    OR name ~* '.*machine.*learning.*'
    OR (name ~ '.*ML.*' AND name !~ '.*HTML.*')
    OR name ~* '.*машинн.*обучен.*')
    AND key_skills ~* '.*python.*';
'''

In [182]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, engine)
df_6_4

,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [80]:
# текст запроса
# подсчет количества ключевых навыков для вакансий DS
# округление до 2 знаков после запятой
# СHR(9) - TAB
query_6_5 = f'''
SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), ''))+1), 2)
FROM
    vacancies
WHERE
    name ~* '.*data.*scientist.*' 
    OR name ~* '.*data.*science.*'
    OR name ~* '.*исследовател.*данн.*'
    OR name ~* '.*machine.*learning.*'
    OR (name ~ '.*ML.*' AND name !~ '.*HTML.*')
    OR name ~* '.*машинн.*обучен.*';
'''

In [181]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, engine)
df_6_5

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [135]:
# текст запроса
# вычисление средней зарплаты DS для каждого типа требуемого опыта
# группировка по опыту работы
query_6_6 = f'''
SELECT
    experience, 
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) as avg_DS_salary 
FROM
    vacancies 
WHERE
    name ~* '.*data.*scientist.*' 
    OR name ~* '.*data.*science.*'
    OR name ~* '.*исследовател.*данн.*'
    OR name ~* '.*machine.*learning.*'
    OR (name ~ '.*ML.*' AND name !~ '.*HTML.*')
    OR name ~* '.*машинн.*обучен.*'
    AND (salary_from is not NULL OR salary_to is not NULL)    
GROUP BY
    experience;
'''

In [180]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, engine)
df_6_6

,experience,avg_ds_salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

**Выводы**

На основе анализа данных в данном юните можно сделать следующие выводы:
* 1771 из 49197 вакансий имеет отношение к анализу данных.
* 51 вакансия представлена для Junior Data Scientist.
* 201 вакансия Data Scientist требует знания SQL или postgres.
* 351 вакансия Data Scientist требует знания Python.
* Примерно 6 ключевых навыков указывается для каждой вакансии DS.
* 243115 руб. - средняя зп для DS с опытом работы от 3 до 6 лет. 139675 руб. - от 1 года до 3 лет. 74643 - без опыта работы. 

# Дополнительное исследование

In [138]:
# текст запроса
# вычисление количества размещенных вакансий для каждого наименование
# группировка по названию
# сортировка по количеству в порядке убывания
# лимит - 10
query_7_1 = f'''
SELECT
    name,
    COUNT(*) AS cnt
FROM
    vacancies 
GROUP BY
    name
ORDER BY
    cnt DESC
LIMIT 10;    
'''

In [139]:
df = pd.read_sql_query(query_7_1, engine)
df

,name,cnt
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


In [183]:
# текст запроса
# вычисление средней зарплаты для каждой вакансии
# группировка по названию вакансии
# сортировка по количеству размещенных вакансий
# лимит - 10
query_7_2 = f'''
SELECT
    v.name,
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))  as avg_salary, 
    COUNT(*) AS cnt
FROM
    vacancies v 
JOIN
    areas a ON v.area_id=a.id
WHERE
    (salary_from IS NOT NULL OR salary_to IS NOT NULL) 
GROUP BY
    v.name
ORDER BY
    cnt DESC
LIMIT 10; 
'''

In [184]:
df_7_2 = pd.read_sql_query(query_7_2, engine)
df_7_2

,name,avg_salary,cnt
0,Системный администратор,50115.0,1287
1,Программист 1С,110189.0,729
2,Специалист технической поддержки,43211.0,335
3,Инженер-программист,50478.0,279
4,Программист 1C,96906.0,228
5,Аналитик,61197.0,197
6,Программист,60817.0,169
7,Специалист службы поддержки в Яндекс Бизнес,26000.0,168
8,Бизнес-аналитик,92228.0,159
9,Инженер технической поддержки,48011.0,140


**Выводы**

На основе анализа данных в данном юните можно сделать следующие выводы:
* Системный администратор - cамая популярная вакансия, размещенная на HH
* Программист 1C - высокооплачивая полулярная позиция

# Общий вывод по проекту

* Общий объем вакансий весьма значителен: почти 50 тысяч предложений в рамках проведенного исследования.
* Среди них выделяются крупные компании, публикующие множество вакансий в более чем 100 различных регионах, где Яндекс занимает лидирующие позиции.
* Наибольшая популярность среди мест работы приходится на Москву и Санкт-Петербург.
* Большая часть работодателей предпочитает предлагать полноценную занятость на полный рабочий день. 
* Более 50% вакансий ориентированы на специалистов со стажем работы от одного до трех лет.
* В половине случаев в объявлениях о вакансиях указывается информация о зарплате, однако чем выше позиция, тем реже встречается такая информация.
* Сфера Data Science не является самой востребованной, но предлагает достаточно высокие зарплаты, даже для начинающих специалистов. Среди ключевых навыков для экспертов в этой области выделяются знание Python и SQL.